In [119]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import timeit
import psutil
import json

weight_external = 0.3
weight_performance = 0.7

In [120]:
#Importing Dataset
reg_dataset = 'https://raw.githubusercontent.com/Akshay-De-Silva/ml_apis/main/bitcoin.csv'
dataset=pd.read_csv(reg_dataset)

dataset = dataset.dropna()

X = dataset[['Open', 'High', 'Low', 'Close', 'Volume_(BTC)', 'Volume_(Currency)']]
y = dataset[['Weighted_Price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

In [121]:
# Linear Regression
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()
training_time = timeit.timeit(lambda: linear_model.fit(X_train_scaled, y_train), number=1)
testing_time = timeit.timeit(lambda: linear_model.predict(X_test_scaled), number=1)

ram_usage_bytes = psutil.virtual_memory().used
ram_usage = ram_usage_bytes / (1024 ** 2)

y_pred = linear_model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

In [122]:
def rmspeFucntion(rmse):
  if np.mean(y_test) == 0:
    return np.nan
  else:
    return (rmse / np.mean(y_test)) * 100

rmspe = rmspeFucntion(rmse)

In [123]:
max_time = 0.024
min_time = 0.008
max_gpu = 2000
min_gpu = 1000
max_cpu = 30
min_cpu = 15
max_ram = 2000
min_ram = 1000

max_rmse = 5.14
min_rmse = 0.19
max_r2 = 1
min_r2 = 0.98

gpu_usage = 1100
cpu_usage = 18

cost = 5000

rmspeci = cost * rmspe

brmseti = (weight_performance * (1-((rmse - min_rmse) / (max_rmse - min_rmse)))) \
        + (weight_external * (1 - ((training_time - min_time) / (max_time - min_time))))

rars_cpu = (weight_performance * ((r2-min_r2)/(max_r2-min_r2)) + (weight_external * (1-((cpu_usage-min_cpu)/(max_cpu-min_cpu)))))
rars_gpu = (weight_performance * ((r2-min_r2)/(max_r2-min_r2)) + (weight_external * (1-((gpu_usage-min_gpu)/(max_gpu-min_gpu)))))
rars_ram = (weight_performance * ((r2-min_r2)/(max_r2-min_r2)) + (weight_external * (1-((ram_usage-min_ram)/(max_ram-min_ram)))))

In [124]:
def reg_to_json(rmspeci, brmseti, rars_cpu, rars_gpu, rars_ram, mse, rmse, mae, r2):
    output = {
        "BATI": 0,
        "RAF1_CPU" : 0,
        "RAF1_GPU" : 0,
        "RAF1_RAM" : 0,
        "F1CI" : 0,
        "BRMSETI" : round(brmseti,2),
        "RARS_CPU" : round(rars_cpu,2),
        "RARS_GPU" : round(rars_gpu,2),
        "RARS_RAM" : round(rars_ram,2),
        "RMSPECI" : round(rmspeci,2),
        "Accuracy" : 0,
        "F1Score" : 0,
        "Precision" : 0,
        "Recall" : 0,
        "MSE" : round(mse,2),
        "RSME" : round(rmse,2),
        "MAE" : round(mae,2),
        "R2" : round(r2,2)
    }
    return json.dumps(output, indent=4)

In [125]:
output = reg_to_json(rmspeci, brmseti, rars_cpu, rars_gpu, rars_ram, mse, rmse, mae, r2)
print(output)

{
    "BATI": 0,
    "RAF1_CPU": 0,
    "RAF1_GPU": 0,
    "RAF1_RAM": 0,
    "F1CI": 0,
    "BRMSETI": 0.24,
    "RARS_CPU": 0.94,
    "RARS_GPU": 0.97,
    "RARS_RAM": 0.98,
    "RMSPECI": 1045.81,
    "Accuracy": 0,
    "F1Score": 0,
    "Precision": 0,
    "Recall": 0,
    "MSE": 0.04,
    "RSME": 0.19,
    "MAE": 0.05,
    "R2": 1.0
}
